![NYC Skyline](img/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many [Airbnb](https://www.airbnb.com/) listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this notebook, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

Our goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:

- What is the average price, per night, of an Airbnb listing in NYC?
- How does the average price of an Airbnb listing, per month, compare to the private rental market?
- How many adverts are for private rooms?
- How do Airbnb listing prices compare across the five NYC boroughs?


In [1]:
#import data 
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
#Load airbnb_price.csv as a dataframe called prices
 
prices = pd.read_csv("data/airbnb_price.csv")

In [3]:
#Step 1: Load airbnb_room_type.xlsx as a dataframe called xls

xls = pd.ExcelFile("data/airbnb_room_type.xlsx")

In [4]:
#Parse the first sheet from xls called room_types

room_types = xls.parse(0)


In [5]:
#Load "data/airbnb_last_review.tsv" as a dataframe called reviews
#pass '\t' to the sep argument of pd.read_csv()

reviews = pd.read_csv("data/airbnb_last_review.tsv", sep='\t') 

In [6]:
#Step 2: Clean the price column to calculate average prices. 

#Update the 'price' column of the prices dataframe
#Replace the words 'pounds' and preceding whitespace from a column replacing with an empty string 

prices["price"] = prices["price"].str.replace(" dollars", "")

In [7]:
#Convert the price column data type to numeric by passing it to the pandas .to_numeric() method.

prices["price"] = pd.to_numeric(prices["price"])


In [8]:
#Step 3: Calculate the average price

# Subset prices for listings costing $0, free_listings
free_listings = prices["price"] == 0


In [9]:
# Update prices by removing all free listings from prices

prices = prices.loc[~free_listings]

In [10]:
# Calculate the average price, round to two decimal places

avg_price = round(prices["price"].mean(), 2)

In [11]:
# Step 4. Comparing costs to the private rental market

#convert the per night prices of the listings into monthly costs, to  compare to the private market.
# Add a new column to the prices DataFrame, price_per_month
prices["price_per_month"] = prices["price"] * 365 / 12

In [12]:
# Calculate average_price_per_month
average_price_per_month = round(prices["price_per_month"].mean(), 2)
difference = round((average_price_per_month - 3100),2)

In [13]:
# Step 5. Cleaning the room_type column

# Convert the room_type column to lowercase
room_types["room_type"] = room_types["room_type"].str.lower()

In [14]:
# Update the room_type column to category data type
room_types["room_type"] = room_types["room_type"].astype("category")

In [15]:
# Create the variable room_frequencies
room_frequencies = room_types["room_type"].value_counts()

In [16]:
# Step 6. What timeframe are we working with?

# Change the data type of the last_review column to datetime
reviews["last_review"] = pd.to_datetime(reviews["last_review"])

In [17]:
# Create first_reviewed, the earliest review date
first_reviewed = reviews["last_review"].dt.date.min()

In [18]:
# Create last_reviewed, the most recent review date
last_reviewed = reviews["last_review"].dt.date.max()

In [19]:
# Step 7. Joining the DataFrames

# Merge prices and room_types to create rooms_and_prices
rooms_and_prices = prices.merge(room_types, how="outer", on="listing_id")

In [20]:
# Merge rooms_and_prices with the reviews DataFrame to create airbnb_merged
airbnb_merged = rooms_and_prices.merge(reviews, how="outer", on="listing_id")

In [21]:
# Drop missing values from airbnb_merged
airbnb_merged.dropna(inplace=True)

In [22]:
# Step 8. Analyzing listing prices by NYC borough

# Extract information from the nbhood_full column and store as a new column, borough
airbnb_merged["borough"] = airbnb_merged["nbhood_full"].str.partition(",")[0]

In [23]:
# Group by borough and calculate summary statistics
boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])

In [24]:
# Round boroughs to 2 decimal places, and sort by mean in descending order
boroughs = boroughs.round(2).sort_values("mean", ascending=False)

In [25]:
# Step 9. Price range by borough

# Create labels for the price range, label_names
label_names = ["Budget", "Average", "Expensive", "Extravagant"]

In [26]:
# Create the label ranges, ranges
ranges = [0, 69, 175, 350, np.inf]

In [27]:
# Insert new column, price_range, into DataFrame
airbnb_merged["price_range"] = pd.cut(airbnb_merged["price"], bins=ranges, labels=label_names)

In [28]:
# Calculate occurence frequencies for each label, prices_by_borough
prices_by_borough = airbnb_merged.groupby(["borough", "price_range"])["price_range"].count()


In [29]:
# Step 10. Storing the final result

solution = {'avg_price': avg_price,
            'average_price_per_month': average_price_per_month,  
            'difference': difference,          
            'room_frequencies': room_frequencies, 
            'first_reviewed': first_reviewed,
            'last_reviewed': last_reviewed,
            'prices_by_borough': prices_by_borough}
print(solution)

{'avg_price': 141.82, 'average_price_per_month': 4313.61, 'difference': 1213.61, 'room_frequencies': entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64, 'first_reviewed': datetime.date(2019, 1, 1), 'last_reviewed': datetime.date(2019, 7, 9), 'prices_by_borough': borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive  